# Interferometry with coherent states using number operator

In [2]:
%matplotlib inline

from math import *
from qutip import *
from pylab import *

from scipy.optimize import minimize as cp_minimize

import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### Symbols & operators

In [3]:
T = sp.symbols('T')
N = 60 # dimension of the Hilbert space

# operators
I = qeye(N) # identity operator
a = destroy(N) # annihilation operator
n = num(N); n_a = tensor(n, I); n_b = tensor(I, n) # number operators
D_n = n_a - n_b # ΔN = n_1 - n_2
S_n = n_a + n_b # ΣN = n_1 + n_2
U = (1j*np.pi/4*(tensor(a.dag(), a) + tensor(a, a.dag()))).expm() # beam splitter operator

# Schwinger representation
Jx = 1/2*(tensor(a.dag(), a) + tensor(a, a.dag()))
Jy = -1j/2*(tensor(a.dag(), a) - tensor(a, a.dag()))
Jz = 1/2*(tensor(a.dag()*a, I) - tensor(I, a.dag()*a))

# Parity operator
Pa = (1j*np.pi*tensor(a.dag()*a,I)).expm()
Pb = (1j*np.pi*tensor(I,a.dag()*a)).expm()
P = (1j*np.pi*a.dag()*a).expm()

### Functions for input states

In [4]:
# returns ρ_in given pure state ψ_in
def rho(psi):
    return psi*psi.dag()

# |ψ_in_1> = |0>|α>
def psi_in_1(alpha):
    return tensor(fock(N, 0),coherent(N, alpha))

# |ψ_in_2> = |β>|β>
def psi_in_2(beta):
    return tensor(coherent(N, beta),coherent(N, beta))
    
# |ψ_in_3> = N(|γδ>+|δγ>)
def psi_in_3(gamma, delta):
    temp = tensor(coherent(N, gamma), coherent(N, delta)) + tensor(coherent(N, delta), coherent(N, gamma))
    return temp.unit()

# |ψ_in_4> = N(a_1†|α_1,α_2>+a_2†|α_1,α_2>)
def psi_in_4(alpha_1, alpha_2):
    psi_alphas = tensor(coherent(N, alpha_1),coherent(N, alpha_2)) # |α_1,α_2>
    temp = tensor(a.dag(), I)*psi_alphas + tensor(I, a.dag())*psi_alphas
    return temp.unit()
    
# |ψ_in_5> = N(a_1†²|α_1,α_2>+a_2†²|α_1,α_2>)
def psi_in_5(alpha_1, alpha_2):
    psi_alphas = tensor(coherent(N, alpha_1), coherent(N, alpha_2)) # |α_1,α_2>
    temp = tensor(a.dag()**2, I)*psi_alphas + tensor(I, a.dag()**2)*psi_alphas
    return temp.unit()

#### Input states 1,2,3

In [ ]:
alpha = 0.75
beta = np.sqrt(alpha**2/2)
gamma = 0.3
delta = 0.750905

rho_in_1 = rho(psi_in_1(alpha))
rho_in_2 = rho(psi_in_2(beta))
rho_in_3 = rho(psi_in_3(gamma, delta))

#### Checking that $\langle N \rangle = \lvert \alpha \rvert^2$

In [ ]:
print('α² =', alpha**2)

print('<N_1> =', np.abs((rho_in_1*S_n).tr()))
print('<N_2> =', np.abs((rho_in_2*S_n).tr()))
print('<N_3> =', np.abs((rho_in_3*S_n).tr()))

### Helper functions

In [5]:
# returns an np-array matrix given Qobj
def Qobj_to_np_array(Qobj):
    M = []
    for i in Qobj:
        M.append(i)
    return np.resize(np.asarray(M), np.shape(Qobj))

# returns the minimum value of expr(var)
def minimizer(expr, var):
    f = sp.lambdify(var, expr)
    res = cp_minimize(f, 1)
    return res.fun

# differentiation using forward difference (fd)
def der_fd_abs(func, d_t):
    der = []
    for i in range(0, len(func)-1):
        der.append(abs((func[i+1]-func[i])/d_t))
    return der

# differentiation using central difference (cd)
def der_cd_abs(func, d_t):
    der = []
    for i in range(1, len(func)-1):
        der.append(abs((func[i+1]-func[i-1])/(2*d_t)))
    return der

## Numerical Schrödinger picture

### Functions returning $\Delta\phi$ using $\hat{J_z}$

In [5]:
def phi_std_dev_num(psi_in, t_min, t_max, d_t):
    D_phi = [] # Δφ
    rho_in = rho(psi_in)
    
    # ΔJz
    Jz_std_dev = sqrt((rho_in*(Jz**2)).tr()-((rho_in*Jz).tr())**2)
    
    # |d<P_a>_out/dφ|
    result = []
    for t in arange(t_min, t_max, d_t):
        F = (1j*t*a.dag()*a).expm() # phase shift operator
        
        psi_out_1 = U*psi_in
        psi_out_mirror = tensor(I,F)*psi_out_1
        psi_out_2 = U*psi_out_mirror
    
        rho_out = rho(psi_out_2)
        result.append((rho_out*Jz).tr())
    
    return Jz_std_dev/der_cd_abs(result, d_t)

#### Caculating & plotting $\Delta\phi_\text{min}$ vs. $\langle n \rangle$ using $\hat{J_z}$

In [6]:
file = open("number_numerical_Schrödinger_1_12.txt","w+")
file.write("# Δφ_min calculated using number operator and numerical Schrödinger method\n")
file.write("# |ψ_in_1> = |0>|α>\n")
file.write("# |ψ_in_2> = |β>|β>\n")
file.write("# ------------------------------------------------------------------------------------------------\n")
file.write("# %-10s %-30s %-10s %-30s\n" % ("alpha","Δφ_min_1","beta","Δφ_min_2"))

86

In [7]:
n_1_avgs = []; phi_1_std_dev_mins = []
n_2_avgs = []; phi_2_std_dev_mins = []

t_min = -0.01; t_max = 2*np.pi; d_t = 0.09
alphas = [0.5*n for n in range (4,14)] # = {2, 2.5, 3 ... 6.5}

for alpha in alphas:
    beta = np.sqrt(alpha**2/2)
    
    rho_in_1 = rho(psi_in_1(alpha))
    rho_in_2 = rho(psi_in_2(beta))
    
    # <n>'s
    n_1_avgs.append(np.abs((rho_in_1*S_n).tr()))
    n_2_avgs.append(np.abs((rho_in_2*S_n).tr()))
    
    # Δφ_min's
    phi_1_std_dev = phi_std_dev_num(rho_in_1, t_min, t_max, d_t)
    min_1 = np.amin(phi_1_std_dev)
    phi_1_std_dev_mins.append(min_1)
    
    phi_2_std_dev = phi_std_dev_num(rho_in_2, t_min, t_max, d_t)
    min_2 = np.amin(phi_2_std_dev)
    phi_2_std_dev_mins.append(min_2)

    file.write("  %-10.8f %-30.25f %-10.8f %-30.25f\n" % (alpha,min_1,beta,min_2))
    print("alpha =", alpha)

file.close()

alpha = 2.0
alpha = 2.5
alpha = 3.0
alpha = 3.5
alpha = 4.0
alpha = 4.5
alpha = 5.0
alpha = 5.5
alpha = 6.0
alpha = 6.5


In [8]:
file = open("number_numerical_Schrödinger_1_345.txt","w+")
file.write("# Δφ_min calculated using number operator and numerical Schrödinger method\n")
file.write("# |ψ_in_3> = N(|α_1,α_2>+|α_2,α_1>)\n")
file.write("# |ψ_in_4> = N(a_1†|α_1,α_2>+a_2†|α_1,α_2>)\n")
file.write("# |ψ_in_5> = N(a_1†²|α_1,α_2>+a_2†²|α_1,α_2>)\n")
file.write("# ------------------------------------------------------------------------------------------------\n")
file.write("# %-10s %-10s %-30s %-30s %-30s\n" % ("alpha_1","alpha_2","Δφ_min_3","Δφ_min_4","Δφ_min_5"))

117

In [1]:
n_3_avgs = []; phi_3_std_dev_mins = []
n_4_avgs = []; phi_4_std_dev_mins = []
n_5_avgs = []; phi_5_std_dev_mins = []

t_min = -0.01; t_max = 2*np.pi; d_t = 0.09
#alpha_1s = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5]
#alpha_2s = [-0.5, -1, -1.5, -2, -2.5, -3, -3.5, -4, -4.5]
alpha_1s = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]
alpha_2s = [-0.5, -1, -1.5, -2, -2.5, -3, -3.5, -4]

for alpha_1,alpha_2 in zip(alpha_1s,alpha_2s):
    rho_in_3 = rho(psi_in_3(alpha_1, alpha_2))
    rho_in_4 = rho(psi_in_4(alpha_1, alpha_2))
    rho_in_5 = rho(psi_in_5(alpha_1, alpha_2))

    # <n>'s
    n_3_avgs.append(np.abs((rho_in_3*S_n).tr()))
    n_4_avgs.append(np.abs((rho_in_4*S_n).tr()))
    n_5_avgs.append(np.abs((rho_in_5*S_n).tr()))

    # Δφ_min's
    phi_3_std_dev = phi_std_dev_num(rho_in_3, t_min, t_max, d_t)
    min_3 = np.amin(phi_3_std_dev)
    phi_3_std_dev_mins.append(min_3)
    
    
    phi_4_std_dev = phi_std_dev_num(rho_in_4, t_min, t_max, d_t)
    min_4 = np.amin(phi_4_std_dev)
    phi_4_std_dev_mins.append(min_4)

    
    phi_5_std_dev = phi_std_dev_num(rho_in_5, t_min, t_max, d_t)
    min_5 = np.amin(phi_5_std_dev)
    phi_5_std_dev_mins.append(min_5)
    
    file.write("  %-10.8f %-10.8f %-30.25f %-30.25f %-30.25f\n" % (alpha_1,alpha_2,min_3,min_4,min_5))
    print("alpha_1 =", alpha_1, ", alpha_2 =", alpha_2)
    
file.close()

NameError: name 'np' is not defined

## Symbolic Heisenberg picture

### Functions returning $\Delta\phi$ using $\hat{J_z}$

In [ ]:
# old version using conversion to numpy arrays and primitive matrix multiplication
def phi_std_dev_0(rho_in, Jx, Jz, Jz_out):
    Jz_std_dev = sp.sqrt(np.dot(rho_in, np.dot(Jz, Jz)).trace() - dot(rho_in, Jz).trace()**2)
    Jz_out_exp = dot(rho_in, Jz_out).trace()
    return Jz_std_dev/abs(sp.diff(Jz_out_exp, T))
# current version using Qobj and optimized multiplication
def phi_std_dev(rho_in):
    Jz_std_dev = np.sqrt((rho_in*Jz**2).tr() - (rho_in*Jz).tr()**2)
    Jz_out_exp = -sp.sin(T)*(rho_in*Jx).tr() + sp.cos(T)*(rho_in*Jz).tr()
    return Jz_std_dev/abs(sp.diff(Jz_out_exp, T))

#### Calculating $\Delta\phi$ and printing minima

In [14]:
phi_1_std_dev = phi_std_dev(rho_in_1)
print('Δφ_1,min =', minimizer(phi_1_std_dev, T))

phi_2_std_dev = phi_std_dev(rho_in_2)
print('Δφ_2,min =', minimizer(phi_2_std_dev, T))

phi_3_std_dev = phi_std_dev(rho_in_3)
print('Δφ_3,min =', minimizer(phi_3_std_dev, T))

Δφ_1,min = 1.3333333333250867
Δφ_2,min = 1.3333333331414607
Δφ_3,min = 1.528576347237865


#### Plotting $\Delta\phi$ vs. $\phi$

In [15]:
plots = sp.plot(phi_1_std_dev, phi_2_std_dev, phi_3_std_dev,
                xlim=[-2*np.pi,2*np.pi], ylim=[0,5],
                xlabel='$\phi$', ylabel='$\Delta\phi$',
                show=False)

plots[0].line_color = 'r'
plots[1].line_color = 'g'
plots[2].line_color = 'b'

# plots.show()

#### Printing all $\Delta\phi_\text{min}$

In [16]:
def print_values(alpha, gammas, deltas):
    beta = np.sqrt(alpha**2/2)

    rho_in_1 = rho(psi_in_1(alpha))
    rho_in_2 = rho(psi_in_2(beta))
    
    phi_1_std_dev = phi_std_dev(rho_in_1)
    print('Δφ_1 =', minimizer(phi_1_std_dev, T))
    
    phi_2_std_dev = phi_std_dev(rho_in_2)
    print('Δφ_2 =', minimizer(phi_2_std_dev, T))
    
    print()

    for gamma,delta in zip(gammas,deltas):
        print('γ =', gamma, ', δ = ', delta)
        
        rho_in_3 = rho(psi_in_3(gamma, delta))

        phi_3_std_dev = phi_std_dev(rho_in_3)
        print('Δφ_3 =', minimizer(phi_3_std_dev, T))
        print()

In [17]:
alpha = 0.75
gammas = [0.1, 0.3, 0.5, 0.7, 0.9]
deltas = [0.874729, 0.750905, 0.560657, 0.357412, 0.00743844]
# print_values(alpha,gammas,deltas)

#### Caculating & plotting $\Delta\phi_\text{min}$ vs. $\langle n \rangle$ using $\hat{J_z}$

In [ ]:
file = open("number_symbolic_Heisenberg_1_12.txt","w+")
file.write("# Δφ_min calculated using number operator and symbolic Heisenberg method\n")
file.write("# |ψ_in_1> = |0>|α>\n")
file.write("# |ψ_in_2> = |β>|β>\n")
file.write("# ------------------------------------------------------------------------------------------------\n")
file.write("# %-10s %-30s %-10s %-30s\n" % ("alpha","Δφ_min_1","beta","Δφ_min_2"))

In [ ]:
n_1_avgs = []; phi_1_std_dev_mins = []
n_2_avgs = []; phi_2_std_dev_mins = []

alphas = [0.5*n for n in range (4,14)]

for alpha in alphas:
    beta = np.sqrt(alpha**2/2)
    
    rho_in_1 = rho(psi_in_1(alpha))
    rho_in_2 = rho(psi_in_2(beta))
    
    # <n>'s
    n_1_avgs.append(np.abs((rho_in_1*S_n).tr()))
    n_2_avgs.append(np.abs((rho_in_2*S_n).tr()))
    
    # Δφ_min's
    phi_1_std_dev = phi_std_dev(rho_in_1)
    min_1 = minimizer(phi_1_std_dev,T)
    phi_1_std_dev_mins.append(min_1)
    
    phi_2_std_dev = phi_std_dev(rho_in_2)
    min_2 = minimizer(phi_2_std_dev,T)
    phi_2_std_dev_mins.append(min_2)
    
    file.write("  %-10.8f %-30.25f %-10.8f %-30.25f\n" % (alpha,min_1,beta,min_2))

In [ ]:
file.close()

In [ ]:
file = open("number_symbolic_Heisenberg_1_345.txt","w+")
file.write("# Δφ_min calculated using number operator and symbolic_Heisenberg method\n")
file.write("# |ψ_in_3> = N(|α_1,α_2>+|α_2,α_1>)\n")
file.write("# |ψ_in_4> = N(a_1†|α_1,α_2>+a_2†|α_1,α_2>)\n")
file.write("# |ψ_in_5> = N(a_1†²|α_1,α_2>+a_2†²|α_1,α_2>)\n")
file.write("# ------------------------------------------------------------------------------------------------\n")
file.write("# %-10s %-10s %-30s %-30s %-30s\n" % ("alpha_1","alpha_2","Δφ_min_3","Δφ_min_4","Δφ_min_5"))

In [ ]:
#alpha_1s = [0.5,1,1,2,2,3,3,4]
#alpha_2s = [0.7,1,2,2,3,3,4,4]
alpha_1s = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5]
alpha_2s = [-0.5, -1, -1.5, -2, -2.5, -3, -3.5, -4, -4.5]

n_3_avgs = []; phi_3_std_dev_mins = []
n_4_avgs = []; phi_4_std_dev_mins = []
n_5_avgs = []; phi_5_std_dev_mins = []

for alpha_1,alpha_2 in zip(alpha_1s,alpha_2s):
    rho_in_3 = rho(psi_in_3(alpha_1, alpha_2))
    rho_in_4 = rho(psi_in_4(alpha_1, alpha_2))
    rho_in_5 = rho(psi_in_5(alpha_1, alpha_2))
    
    # <n>'s
    n_3_avgs.append(np.abs((rho_in_3*S_n).tr()))
    n_4_avgs.append(np.abs((rho_in_4*S_n).tr()))
    n_5_avgs.append(np.abs((rho_in_5*S_n).tr()))
    
    # Δφ_min's
    phi_3_std_dev = phi_std_dev(rho_in_3)
    min_3 = minimizer(phi_3_std_dev,T)
    phi_3_std_dev_mins.append(min_3)
    
    phi_4_std_dev = phi_std_dev(rho_in_4)
    min_4 = minimizer(phi_4_std_dev,T)
    phi_4_std_dev_mins.append(min_4)
    
    phi_5_std_dev = phi_std_dev(rho_in_5)
    min_5 = minimizer(phi_5_std_dev,T)
    phi_5_std_dev_mins.append(min_5)
    
    file.write("  %-10.8f %-10.8f %-30.25f %-30.25f %-30.25f\n" % (alpha_1,alpha_2,min_3,min_4,min_5))
    print("alpha_1 =", alpha_1, ", alpha_2 =", alpha_2)

In [1]:
file.close()

NameError: name 'file' is not defined

## Schrödinger picture

In [1]:
D_n_avg = []
D_phi = []

for t in arange(-2*np.pi-1,2*np.pi,0.1) : # t (as in θ): angle of phase shifter
    F = (1j*t*a.dag()*a).expm() # phase shift operator
    
    psi_out_1 = U*psi_in_1
    psi_out_mirror = tensor(I,F)*psi_out_1
    psi_out_2 = U*psi_out_mirror
    
    rho_out = psi_out_2*psi_out_2.dag() # density matrix: ρ = |ψ_out_2><ψ_out_2|
    
    # for <ΔN>
    D_n_avg.append((rho_out*D_n).tr()) # <ΔN> = ρ*ΔN
    
    # for Δθ
    Jz = 1/2*D_n # Jz = (1/2)ΔN
    D_Jz = sqrt((rho_out*(Jz**2)).tr()-((rho_out*Jz).tr())**2)
    d_Jz_out_d_phi = abs(1/2*(np.sin(t)*alpha**2))
    
    D_phi.append(D_Jz/d_Jz_out_d_phi)

D_n_avg_array = np.array(D_n_avg)
D_phi_array = np.array(D_phi)

NameError: name 'arange' is not defined

In [16]:
x = arange(-2*np.pi-1,2*np.pi,0.1)

In [17]:
plt.plot(x,D_n_avg_array,'m',label=r'$\Delta N$')

plt.xlabel(r'$\theta$')
plt.ylabel(r'$\langle\Delta\hat{N}\rangle$')

plt.legend(loc='lower right')
plt.close()

In [18]:
plt.plot(x,D_phi_array,'c',label=r'$\Delta \phi$')

plt.xlabel(r'$\theta$')
plt.ylabel(r'$\Delta\phi$')
plt.ylim(-1,20)

plt.legend(loc='best')
plt.close()